# T1036 - Masquerading
Adversaries may attempt to manipulate features of their artifacts to make them appear legitimate or benign to users and/or security tools. Masquerading occurs when the name or location of an object, legitimate or malicious, is manipulated or abused for the sake of evading defenses and observation. This may include manipulating file metadata, tricking users into misidentifying the file type, and giving legitimate task or service names.

Renaming abusable system utilities to evade security monitoring is also a form of [Masquerading](https://attack.mitre.org/techniques/T1036).(Citation: LOLBAS Main Site)

## Atomic Tests

### Atomic Test #1 - System File Copied to Unusual Location
It may be suspicious seeing a file copy of an EXE in System32 or SysWOW64 to a non-system directory or executing from a non-system directory.
**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
copy-item "$env:windir\System32\cmd.exe" -destination "$env:allusersprofile\cmd.exe"
start-process "$env:allusersprofile\cmd.exe"
sleep -s 5 
stop-process -name "cmd" | out-null```

In [ ]:
Invoke-AtomicTest T1036 -TestNumbers 1

### Atomic Test #2 - Malware Masquerading and Execution from Zip File
When the file is unzipped and the README.cmd file opened, it executes and changes the .pdf to .dll and executes the dll. This is a BazaLoader technique [as reported here](https://twitter.com/ffforward/status/1481672378639912960)
**Supported Platforms:** windows
#### Dependencies:  Run with `None`!
##### Description: Zip file must be present.
##### Check Prereq Commands:
```None
if (Test-Path $env:userprofile\Downloads\T1036.zip) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```None
Invoke-WebRequest -OutFile "$env:userprofile\Downloads\T1036.zip" https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1036/bin/T1036.zip

```

In [ ]:
Invoke-AtomicTest T1036 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Expand-Archive -Path $env:userprofile\Downloads\T1036.zip -DestinationPath $env:userprofile\Downloads\T1036 -Force
cd $env:userprofile\Downloads\T1036
cmd /c $env:userprofile\Downloads\T1036\README.cmd >$null 2>$null```

In [ ]:
Invoke-AtomicTest T1036 -TestNumbers 2

## Detection
Collect file hashes; file names that do not match their expected hash are suspect. Perform file monitoring; files with known names but in unusual locations are suspect. Likewise, files that are modified outside of an update or patch are suspect.

If file names are mismatched between the file name on disk and that of the binary's PE metadata, this is a likely indicator that a binary was renamed after it was compiled. Collecting and comparing disk and resource filenames for binaries by looking to see if the InternalName, OriginalFilename, and/or ProductName match what is expected could provide useful leads, but may not always be indicative of malicious activity. (Citation: Elastic Masquerade Ball) Do not focus on the possible names a file could have, but instead on the command-line arguments that are known to be used and are distinct because it will have a better rate of detection.(Citation: Twitter ItsReallyNick Masquerading Update)

Look for indications of common characters that may indicate an attempt to trick users into misidentifying the file type, such as a space as the last character of a file name or the right-to-left override characters"\u202E", "[U+202E]", and "%E2%80%AE”.

## Shield Active Defense
### Behavioral Analytics 
 Deploy tools that detect unusual system or user behavior.
 

 Instrument a system to collect detailed information about process execution and user activity, develop a sense of normal or expected behaviors, and alert on abnormal or unexpected activity.  This can be accomplished either onboard the target system or by shipping data to a centralized analysis and alerting system.
#### Opportunity
There is an opportunity to detect the presence of an adversary by identifying and alerting on anomalous behaviors.
#### Use Case
A defender can look for known files in non-standard locations or files that are creating anomalous processes or connections.
#### Procedures
Use behavioral analytics to detect Living Off The Land Binaries (LOLBins) being used to download and execute a file.
Use behavioral analytics to identify a system running development tools, but is not used by someone who does development.
Use behavioral analytics to identify abnormal system processes being used to launch a different process.